# Game to Movie Video Conversion

## Main Imports

In [6]:
from os import listdir
import cv2
import random
import natsort
import numpy as np
from matplotlib import pyplot as plt

from keras.preprocessing.image import load_img, img_to_array

## Preprocess the video data
### Function to read frames from files

In [31]:
def read_frames(video_path, save_path, problem_frames, start_frame, end_frame, frame_skip, counter):
    
    # initialise frame capturing
    vidcap = cv2.VideoCapture(video_path)
    vidcap.set(1, start_frame)
    success, image = vidcap.read()
    frame_number = start_frame
    
    while success and frame_number < end_frame:

        # save frames as images, capturing every frame_skip frames
        cv2.imwrite(save_path + '%d.jpg' % counter, image)
        frame_number += frame_skip
        counter += 1
        
        if frame_number in problem_frames:
            original_frame_number = frame_number
            random_frame = random.randint(start_frame, end_frame)
            vidcap.set(1, random_frame)
            frame_number = original_frame_number
        else:
            vidcap.set(1, frame_number)
        success, image = vidcap.read()

### Initialise file paths

In [33]:
game_load_path = 'data/game/MafiaVideogame.mp4'
game_save_path = 'data/game/frames/'
game_problem_frames = [6510, 6698, 11774, 11962, 15158, 20798, 22490, 23054, 29070, 34334, 34522, 41290, 41478, 49562, 49750, 
                       65730, 65918, 66106, 69302, 73438, 73626, 74002, 80018, 80206, 87162, 89230, 92426, 96938, 103518,
                      103706, 103894, 116302, 130590, 130778, 138298, 138486, 142058, 147510, 147698, 154654, 154842, 157662,
                      166686, 166874, 173642, 173830, 180410, 192066, 192254, 196578, 202406, 202594, 207106, 214626, 221582,
                      221770, 228914, 229102]

movies_folder = 'data/movie/'
movie_save_path = 'data/movie/frames/'

# data holding start frame, end frame, frame skip
movie_data = {
    "TheGodfather.mp4": (204, 12204, 30, [1074, 1104, 1134, 1164, 1194]),
    "TheIrishman.mp4": (75, 23275, 58, [12603, 12661, 12719]),
    "TheSopranos.mp4": (80, 51280, 128, [9424, 11472, 27984, 28112, 28240, 48848])
}

### Read frames from game file

In [20]:
read_frames(game_load_path, game_save_path, game_problem_frames, 6510, 232110, 188, counter=1)

### Read frames from movie files

In [35]:
counter = 1
for movie_name, frame_data in movie_data.items():
    
    movie_load_path = movies_folder + movie_name
    start_frame, end_frame, frame_skip, movie_problem_frames = frame_data[0], frame_data[1], frame_data[2], frame_data[3]
    read_frames(movie_load_path, movie_save_path, movie_problem_frames, start_frame, end_frame, frame_skip, counter)
    counter += 400

### Set train/validation/test sizes and paths

In [46]:
game_train_path = 'data/game_train/'
game_val_path = 'data/game_val/'
game_test_path = 'data/game_test/'

movie_train_path = 'data/movie_train/'
movie_val_path = 'data/movie_val/'
movie_test_path = 'data/movie_test/'

data_length = len(listdir(game_save_path))
train_size = int(0.8 * data_length)
val_size = int(0.2 * train_size)
train_size -= val_size
test_size = int(0.2 * data_length)

game_data = natsort.natsorted(listdir(game_save_path))
movie_data = natsort.natsorted(listdir(movie_save_path))

384 96 120


### Function to split the data into folders

In [47]:
def split_data(size, data, save_path, new_save_path):
    
    for i in range(size[0], size[1]):

        image_name = data[i]
        image_path = save_path + image_name

        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        cv2.imwrite(new_save_path + image_name, image)

### Splitting the data

In [50]:
split_data((0, train_size), game_data, game_save_path, game_train_path)
split_data((train_size, train_size + val_size), game_data, game_save_path, game_val_path)
split_data((train_size + val_size, data_length), game_data, game_save_path, game_test_path)

split_data((0, train_size), movie_data, movie_save_path, movie_train_path)
split_data((train_size, train_size + val_size), movie_data, movie_save_path, movie_val_path)
split_data((train_size + val_size, data_length), movie_data, movie_save_path, movie_test_path)

In [2]:
def load_images(path, size=(256, 256)):
    
    data_list = list()
    
    # enumerate filenames in directory, assume all are images
    for filename in listdir(path):
        
        # load and resize the image
        pixels = load_img(path + filename, target_size=size)
        pixels = img_to_array(pixels)
        data_list.append(pixels)
    
    return np.asarray(data_list)

In [3]:
# load dataset A
game_data = load_images('data/game_train/')
print('Loaded game data:', game_data.shape)

# load dataset B
movie_data = load_images('data/movie_train/')
print('Loaded movie data:', movie_data.shape)

Loaded game data: (384, 256, 256, 3)
Loaded movie data: (384, 256, 256, 3)
Saved dataset


In [4]:
game_data = (game_data - 127.5) / 127.5
movie_data = (movie_data - 127.5) / 127.5